In [1]:
## Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from ipywidgets.embed import embed_minimal_html

## Import API key
from api_keys import g_key

In [2]:
## configure the gmaps key
gmaps.configure(api_key=g_key)

In [3]:
## load up reactor data
usReactors = pd.read_excel('../Data/LatLongPlants.xlsx')
usReactors.head()

,Name,City,State,Lat,Long,Open,License Expires,Unnamed: 7,Unnamed: 8,Unnamed: 9,Website
0,"Arkansas Nuclear One, Unit 1",London,AR,35.330898,-93.247162,1974,2034,NaN,NaN,NaN,https://www.nrc.gov/info-finder/reactors/
1,"Arkansas Nuclear One, Unit 2",London,AR,35.330898,-93.247162,1974,2034,NaN,NaN,NaN,NaN
2,Beaver Valley Power Station 1,Shippingport,PA,40.629372,-80.421249,1976,2036,NaN,NaN,NaN,NaN
3,Beaver Valley Power Station 2,Shippingport,PA,40.629372,-80.421249,1976,2036,NaN,NaN,NaN,NaN
4,Braidwood Station 1,Braceville,IL,41.226929,-88.264282,1987,2046,NaN,NaN,NaN,NaN


In [4]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Reactor Name</dt><dd>{Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>State</dt><dd>{State}</dd>
</dl>
"""

In [5]:
## make dfs that contain reactors according decade build
def reactorDecade(df, decadeStart, decadeEnd):
    #decadeDF = df[(df.Open >= decadeStart) & (df.Open < decadeEnd)]
    decadeDF = df[(df.Open < decadeEnd)]
    # reactorLocations = decadeDF[["Lat", "Long"]]
    
    # Using the template add the hotel marks to the heatmap
    info_box_template = """
    <dl>
    <dt>Reactor Name</dt><dd>{Name}</dd>
    <dt>City</dt><dd>{City}</dd>
    <dt>State</dt><dd>{State}</dd>
    </dl>
    """    
    
    
    reactor_info = [info_box_template.format(**row) for index, row in decadeDF.iterrows()]
    reactorLocations = decadeDF[["Lat", "Long"]]
    

    markerLayer = gmaps.marker_layer(reactorLocations, info_box_content = reactor_info)
    return markerLayer, reactorLocations

In [25]:
## get decade-specific reactor markers
markerLayer1960, reactorLocations1960 = reactorDecade(usReactors, 1960, 1970)
markerLayer1970, reactorLocations1970 = reactorDecade(usReactors, 1970, 1980)
markerLayer1980, reactorLocations1980 = reactorDecade(usReactors, 1980, 1990)
markerLayer1990, reactorLocations1990 = reactorDecade(usReactors, 1990, 2000)
markerLayer2000, reactorLocations2000 = reactorDecade(usReactors, 2000, 2010)
markerLayer2010, reactorLocations2010 = reactorDecade(usReactors, 2010, 2020)


In [26]:
# make loop to make population layers
def popLayer(popFile):
    popDF = pd.read_csv(f"../Data/{popFile}.csv")
    cityLocations = popDF[["Latitude", "Longitude"]]
    cityPops = popDF.Population.values
    maxPop = cityPops[0].astype(float)
    pop_layer = gmaps.heatmap_layer(cityLocations, weights=cityPops, dissipating=False, max_intensity=maxPop, point_radius=2)
    return pop_layer

In [27]:
heatPop1940 = popLayer('cityCensus_1940')
heatPop1950 = popLayer('cityCensus_1950')
heatPop1960 = popLayer('cityCensus_1960')
heatPop1970 = popLayer('cityCensus_1970')
heatPop1980 = popLayer('cityCensus_1980')
heatPop1990 = popLayer('cityCensus_1990')
heatPop2000 = popLayer('cityCensus_2000')
heatPop2010 = popLayer('cityCensus_2010')

In [14]:
fig1960 = gmaps.figure()
fig1960.add_layer(markerLayer1960)
fig1960.add_layer(heatPop1960)
fig1960

Figure(layout=FigureLayout(height='420px'))

In [15]:
fig1970 = gmaps.figure()
fig1970.add_layer(markerLayer1970)
fig1970.add_layer(heatPop1970)
fig1970

Figure(layout=FigureLayout(height='420px'))

In [16]:
fig1980 = gmaps.figure()
fig1980.add_layer(markerLayer1980)
fig1980.add_layer(heatPop1980)
fig1980

Figure(layout=FigureLayout(height='420px'))

In [17]:
fig1990 = gmaps.figure()
fig1990.add_layer(markerLayer1990)
fig1990.add_layer(heatPop1990)
fig1990

Figure(layout=FigureLayout(height='420px'))

In [18]:
fig2000 = gmaps.figure()
fig2000.add_layer(markerLayer2000)
fig2000.add_layer(heatPop2000)
fig2000

Figure(layout=FigureLayout(height='420px'))

In [19]:
fig2010 = gmaps.figure()
fig2010.add_layer(markerLayer2010)
fig2010.add_layer(heatPop2010)
fig2010

Figure(layout=FigureLayout(height='420px'))

AttributeError: 'Figure' object has no attribute 'savefig'